In [134]:
import numpy
import pandas
from abc import ABCMeta, abstractmethod

class MachineLearning(metaclass=ABCMeta):
    
    def __init__(self, x, y, tts, thresh):
        if len(x) != len(y):
            raise ValueError("X and Y should have the same amount of samples")
        if type(x) != numpy.ndarray:
            x = numpy.array(x)
        self.xtrain, self.ytrain, self.xtest, self.ytest = self.tts(x, y, tts, thresh)
        self.num_samples = len(self.xtrain)
        self.num_features = len(self.xtrain[0])
            
        
    def get_train(self):
        zipped_ = zip([i for i in range(len(self.xtrain))], self.xtrain.transpose())
        df_data = {}
        for n, sample in zipped_:
            df_data["feature"+str(n)] = sample
        return pandas.DataFrame(df_data)
            
    
    def get_test(self):
        testd = {}
        if self.xtest is None:
            raise RuntimeWarning("No Test Data")
        else:
            testd["x_test"] = self.xtest
            if self.ytest is not None:
                testd["y_test"] = self.ytest
    
    def tts(self, x, y, tts = 0.8, thresh = 10):
        xtrain, ytrain, xtest, ytest = None, None, None, None
        xlen = len(x)
        train_len = xlen * tts
        test_len = xlen - train_len
        if y is not None:
            if thresh > test_len:
                xtrain, xtest = x.copy(), x.copy()
                ytrain, ytest = y.copy(), y.copy()
            else:
                xtrain, xtest = x.copy()[0:train_len], x.copy()[train_len:]
                ytrain, ytest = y.copy()[0:train_len], y.copy()[train_len:]
        else:
            xtrain, xtest = x.copy(), x.copy()
        return xtrain, ytrain, xtest, ytest
        
    @abstractmethod
    def fit(self):
        pass
    
    @abstractmethod
    def predict(self, x):
        pass
        
    
    def test(self):
        predictions = predict(self.xtest)
        return pandas.DataFrame({"x": self.xtest,
                                "predictions": predictions,
                                 "Actual": self.ytest })

In [135]:
class kNN(MachineLearning):
    
    def __init__(self, x, y, tts=0.8, thresh=10):
        super(kNN, self).__init__(x, y, tts=0.8, thresh=10)
        
    def fit(self, x, y=None):
        pass
    
    def predict(self, x):
        pass
        

In [136]:
x = kNN([[1, 1], [2,2], [3,3]], ['low', 'low', 'high'])